<div style="text-align:right">Update date: Feb 22, 2024</div><br>

# Code execution<br>
This notebook contains the execution code to generate the scoring of each potential client. Additionally, the validation dataset was included with the scoring returned by the model to later build a dashboard in Power BI.

In [69]:
from os import strerror
import cloudpickle
import pandas as pd

# Load data
df = pd.read_csv('data/validation.csv', index_col='id', sep=",")

# Data quality
df.drop_duplicates(inplace=True)
condicion = ((df['no_enviar_email'] != 'Yes')
             & (df['no_llamar'] != 'Yes')
             & (df['ult_actividad'] != 'Email Bounced')
            )
df = df[condicion]
df_all_columns = df.copy()

final_vars = [
        'ambito',
        'descarga_lm',
        'fuente',
        'ocupacion',
        'origen',
        'score_actividad',
        'score_perfil',
        'tiempo_en_site_total',
        'ult_actividad',
        'visitas_total'
]

df = df[final_vars]

try:
    with open('models/pipe_execution.pickle', mode='rb') as file:
        pipe_ejecucion = cloudpickle.load(file)
except IOError as e:
    print('I/O error in file(verificar ruta):', strerror(e))

scoring = pipe_ejecucion.predict_proba(df)[:, 1]


### Generate file for the marketing and sales team dashboard

#### Data quality

Imputation of null values, treatment of outliers and transformations in the data: in the same sense in which it was carried out to generate the prediction.

In [70]:
# Update values in variables:
var_impute_value = [
    'fuente',
    'ult_actividad',
]
var_impute_mode = ['ocupacion', 'ambito']
# Define helper functions
def impute_mode(variable):
    return variable.fillna(variable.mode()[0])

def impute_median(variable):
    if pd.api.types.is_integer_dtype(variable):
        return variable.fillna(int(variable.median()))
    
    return variable.fillna(variable.median())

def group_rare_categories(variable, criterio=0.05):
    frequency = variable.value_counts(normalize=True)
    temp = list(frequency.loc[frequency < criterio].index.values)
    temp2 = np.where(variable.isin(temp), 'OTROS', variable)
    
    return temp2

def data_quality(df):
    temp = df.astype({'visitas_total': 'Int64'})             
    temp[var_impute_mode] = temp[var_impute_mode].apply(impute_mode)
    temp[var_impute_value] = temp[var_impute_value].fillna('DESCONOCIDO')
    var_impute_median = temp.select_dtypes(include='number').columns.to_list()
    temp[var_impute_median] = temp[var_impute_median].apply(impute_median)
    vars_rare = temp.select_dtypes(exclude='number').columns.to_list()   
    for variable in vars_rare:
        temp[variable] = group_rare_categories(temp[variable], criterio=0.02)
    #Winsorization manual
    temp['visitas_total'] = temp['visitas_total'].clip(0, 50)
    temp['paginas_vistas_visita'] = temp['paginas_vistas_visita'].clip(0, 20)
    
    return(temp)

In [71]:
dashboard_data = data_quality(df_all_columns)

#### Generate business variable

Add scoring variable

In [76]:
dashboard_data['scoring'] = scoring.copy()
dashboard_data

,origen,fuente,no_enviar_email,no_llamar,compra,visitas_total,tiempo_en_site_total,paginas_vistas_visita,ult_actividad,ambito,...,conociste_google,conociste_revista,conociste_periodico,conociste_youtube,conociste_facebook,conociste_referencias,score_actividad,score_perfil,descarga_lm,scoring
id,,,,,,,,,,,,,,,,,,,,,
598572,Landing Page Submission,Google,No,No,1,5,57,2.50,Email Opened,Finance Management,...,No,No,No,No,No,No,14.0,16.0,No,0.548837
611369,API,Chat,No,No,0,0,0,0.00,Email Opened,Select,...,No,No,No,No,No,No,14.0,16.0,No,0.126494
639460,Landing Page Submission,Google,No,No,0,6,422,6.00,SMS Sent,Human Resource Management,...,No,No,No,No,No,No,13.0,16.0,No,0.206618
658470,API,Chat,No,No,0,0,0,0.00,Chat Conversation,Select,...,No,No,No,No,No,No,17.0,15.0,No,0.244469
633876,Landing Page Submission,Organic Search,No,No,0,7,945,3.50,Email Opened,OTROS,...,No,No,No,No,No,No,14.0,13.0,Yes,0.222110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632936,Landing Page Submission,Google,No,No,1,16,1253,3.20,Email Link Clicked,"Banking, Investment And Insurance",...,No,No,No,No,No,No,16.0,19.0,No,0.918615
635574,Landing Page Submission,Direct Traffic,No,No,0,4,4,2.00,Email Opened,OTROS,...,No,No,No,No,No,No,14.0,14.0,Yes,0.047413
629260,Landing Page Submission,Google,No,No,1,4,156,1.33,SMS Sent,International Business,...,No,No,No,No,No,No,14.0,16.0,No,0.244613


Add segment variable

In [100]:
segment = pd.read_pickle('data/validation_segment.pickle')

In [99]:
dashboard_data = dashboard_data.merge(
    segment,
    left_on=dashboard_data.index,
    right_on='id'
)

Lead discretization

Based on the specific objectives of the business, the cuts for scoring are established.

In [129]:
cuts = [0, 0.70, 0.80, 1]
names = ['Cold', 'Medium', 'Hot']
dashboard_data['lead_category'] = pd.qcut(dashboard_data.scoring, cuts, labels=names)

Save dataset

In [155]:
dashboard_data.to_csv('data/dashboard_data.csv')